<a href="https://colab.research.google.com/github/MoisesGuerreroUdeA/Seminario-Analitica-CDatos/blob/dataset-ren-energy/jupyter/ME03/ME03_dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de inteligencia artificial para mejorar el desempeño de un proyecto de generación fotovoltaica

## Seminario de Analítica y Ciencia de Datos

Moisés Alfonso Guerrero Jiménez, Andrés Castaño Licona

### Descripción Datos

Tal como se indicó en entregas previas, se realizó un acercamiento con un funcionario de EPM perteneciente a la Unidad de Hidrometría y Calidad Generación, correspondiente al área responsable del gobierno de datos del proyecto Tepuy y otros relativos a la generación de energía.

De acuerdo a la información suministrada por el funcionario, la fuente de información corresponde a datos medidos en una estación meteorológica ubicada en la zona de actual instalación del proyecto en el municipio de La Dorada, Caldas.

Las variables medidas en la estación que nos fueron informadas, corresponden a:

* Irradiación solar $[W/m^2]$
* Temperatura $[°C]$
* Humedad relativa $[\%]$
* Precipitación
* Velocidad del viento
* Dirección del viento

Dado que actualmente no ha sido posible aún obtener acceso al conjunto de datos requerido, se realizará un análisis y preprocesamiento haciendo uso de un dataset disponible en *Kaggle*, que cuenta con variables similares a las ya mencionadas. Este preprocesamiento y análisis podrá ser aplicado de manera muy similar al dataset del proyecto Tepuy, una vez se tenga el acceso requerido.

El dataset mencionado está disponible en el enlace https://www.kaggle.com/datasets/dronio/SolarEnergy, lleva por nombre *Solar Radiation Prediction* y contiene datos meteorológicos del HI-SEAS en Hawaii, una estación de simulación espacial análoga en un espacio aislado similar a la superficie de Marte o la Luna, situado cerca al volcán Mauna Loa.

Los datos de este dataset incluyen:

* Irradiación solar $[W/m^2]$
* Temperatura $[°F]$
* Presión barométrica $[Hg]$
* Humedad $[\%]$
* Dirección del viento $[°]$
* Velocidad del viento $[mph]$
* Hora de salida/puesta del sol.

## Lectura de los datos

Se realiza la lectura de los datos directamente por medio de la API de Kaggle, haciendo uso de un archivo de credenciales `kaggle.json`, el cual debe ser incluido dentro del entorno de ejecución del Colab al momento de ejecutar el notebook.

In [2]:
# Importamos la librería json requerida para leer el archivo de credenciales
import json

# Como prerrequisito es necesario tener un archivo de credenciales kaggle.json en el directorio del notebook
file = open('kaggle.json',"r")
api_token = json.load(file)

In [3]:
# Creamos el directorio de .kaggle dentro del entorno de Colab
! mkdir /root/.kaggle

# Agregamos el archivo kaggle.json vacío
! touch /root/.kaggle/kaggle.json

In [4]:
# Abrimos el archivo y escribimos el token
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

# Asignamos permisos para acceder al archivo de credenciales
! chmod 600 /root/.kaggle/kaggle.json

In [5]:
# Instalamos kaggle CLI
! pip install kaggle

# Descargamos el Dataset con la API de kaggle
! kaggle datasets download -d dronio/SolarEnergy

# Descomprimimos el archivo .zip descargado
! unzip -q SolarEnergy.zip

  0% 0.00/511k [00:00<?, ?B/s]
100% 511k/511k [00:00<00:00, 61.7MB/s]


El archivo descargado y descomprimido se encuentra en formato CSV con nombre `SolarPrediction.csv`

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Se realiza la lectura del archivo CSV
df = pd.read_csv('SolarPrediction.csv', sep = ',', header = 0)

In [8]:
df

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [ ]:
df.dtypes

UNIXTime                    int64
Data                       object
Time                       object
Radiation                 float64
Temperature                 int64
Pressure                  float64
Humidity                    int64
WindDirection(Degrees)    float64
Speed                     float64
TimeSunRise                object
TimeSunSet                 object
dtype: object

In [ ]:
df.describe(include='all')

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
count,3.268600e+04,32686,32686,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686,32686
unique,NaN,118,8299,NaN,NaN,NaN,NaN,NaN,NaN,51,56
top,NaN,12/1/2016 12:00:00 AM,16:20:18,NaN,NaN,NaN,NaN,NaN,NaN,06:11:00,17:42:00
freq,NaN,288,24,NaN,NaN,NaN,NaN,NaN,NaN,1399,3682
mean,1.478047e+09,NaN,NaN,207.124697,51.103255,30.422879,75.016307,143.489821,6.243869,NaN,NaN
std,3.005037e+06,NaN,NaN,315.916387,6.201157,0.054673,25.990219,83.167500,3.490474,NaN,NaN
min,1.472724e+09,NaN,NaN,1.110000,34.000000,30.190000,8.000000,0.090000,0.000000,NaN,NaN
25%,1.475546e+09,NaN,NaN,1.230000,46.000000,30.400000,56.000000,82.227500,3.370000,NaN,NaN
50%,1.478026e+09,NaN,NaN,2.660000,50.000000,30.430000,85.000000,147.700000,5.620000,NaN,NaN
75%,1.480480e+09,NaN,NaN,354.235000,55.000000,30.460000,97.000000,179.310000,7.870000,NaN,NaN
